<a href="https://colab.research.google.com/github/hackerpranavpandey/DataBricks-LLM-Course---NLP-Tasks/blob/main/tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Auto loaders for tokenizers and models**

We have already seen the dataset and pipeline abstractions from Hugging Face.While a pipeline is a quick way to set up an LLM for agiven task,the slightly lower0level abstraction model and tokenizer pemit a bit more control over the options.

Given an input

Tokenize them means convert them to token indices

Apply model on this tokenized dats

Decode the summaries

API docs:

AutoTokenizer

AutoModelForSeq2SeqLM


In [2]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
tokenizer=AutoTokenizer.from_pretrained("t5-small")  ## load pretrained tokenizer as well as model
model=AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [3]:
%pip install datasets
from datasets import load_dataset
xsum_dataset=load_dataset(
    "xsum",
    version="1.2.0"
)

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [4]:
xsum_samples=xsum_dataset["train"][:10]
articles=list(map(
    lambda article:"summarizer" + article,  ## need to prepare prefix summarizer as prompt
    xsum_samples["document"]
))

In [5]:
import pandas as pd
display(pd.DataFrame(articles,columns=["prompts"]))

,prompts
0,summarizerThe full cost of damage in Newton St...
1,summarizerA fire alarm went off at the Holiday...
2,summarizerFerrari appeared in a position to ch...
3,"summarizerJohn Edward Bates, formerly of Spald..."
4,summarizerPatients and staff were evacuated fr...
5,summarizerSimone Favaro got the crucial try wi...
6,"summarizerVeronica Vanessa Chango-Alverez, 31,..."
7,summarizerBelgian cyclist Demoitie died after ...
8,"summarizerGundogan, 26, told BBC Sport he ""can..."
9,summarizerThe crash happened about 07:20 GMT a...


In [6]:
# Tokenize the input
inputs = tokenizer(articles, max_length=1024, return_tensors="pt", padding=True, truncation=True)
print("input_ids:")
print(inputs["input_ids"])
print("attention_mask:")
print(inputs["attention_mask"])

input_ids:
tensor([[21603,    52,   634,  ...,     0,     0,     0],
        [21603,    52,   188,  ...,     0,     0,     0],
        [21603,    52,   371,  ...,    12,   103,     1],
        ...,
        [21603,    52,  2703,  ...,     0,     0,     0],
        [21603,    52,   517,  ...,     0,     0,     0],
        [21603,    52,   634,  ...,     0,     0,     0]])
attention_mask:
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [7]:
## generating summary now
summary_id= model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    num_beams=4,
    min_length=0,
    max_length=40,
)
print(summary_id)

tensor([[    0,     8,   423,   583,    13,  1783,    16, 20126, 16496,    19,
           341,   271, 14841,     3,     5,   186,  7540,    16,   158,    15,
          2296,     7,  5718,  2367, 14621,  4161,    57,  4125,   387,     3,
             5,     3,     9,  8347,  5685,  3048,    16,   286,   640,     8],
        [    0,  1472,  6196,   877,   326,    44,     8,  9108,    86,    29,
            16,  6000,  1887,    44,    81, 11484,    10,  1755,   272,  4209,
            30,  1856,     3,     5,  2554,   130,  1380,    12,  1175,     8,
          1595,    38,    79,     3,  9094,  1067,     3,     5,    80,    13],
        [    0,     3,   849,  2239,     7, 14014,     3,     9,     3,    60,
          8234,   232,   227,     3, 19585,   643,   243,    96,    29,    32,
           964,  8033,    47,   787,    30,   213,     3,    88,   225,  2447,
           121,  4049, 11968,    29,    15,    91,    18, 23058,   372,    18],
        [    0,     8,     3,  3708,    18,  1201

In [8]:
## now lets decode the generated summaries
decode_summaries= tokenizer.batch_decode(summary_id, skip_special_tokens=True)
display(pd.DataFrame(decode_summaries,columns=["decoded_summaries"]))

,decoded_summaries
0,the full cost of damage in Newton Stewart is s...
1,fire alarm went off at the Holiday Inn in Hope...
2,stewards handed a reprimand after governing bo...
3,the 67-year-old is accused of committing the o...
4,a man receiving psychiatric treatment at the c...
5,Gregor Townsend gave a debut to powerhouse fij...
6,"Veronica Vanessa Chango-Alverez, 31, was kille..."
7,the 25-year-old was hit by a motorbike during ...
8,"rGundogan says he ""can see the finishing line""..."
9,"the man, aged in his 20s, was treated at the s..."


  Model-specific tokenizer and model loaders

You can also more directly load specific tokenizer and model types, rather than relying on `Auto*` classes to choose the right ones for you.

 MAGIC API docs:

 [T5Tokenizer](https://huggingface.co/docs/transformers/main/en/model_doc/t5#transformers.T5Tokenizer)

 [T5ForConditionalGeneration](https://huggingface.co/docs/transformers/main/en/model_doc/t5#transformers.T5ForConditionalGeneration)


In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained(
    "t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# The tokenizer and model can then be used similarly to how we used the ones loaded by the Auto* classes.
inputs = tokenizer(
    articles, max_length=1024, return_tensors="pt", padding=True, truncation=True
)
summary_ids = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    num_beams=2,
    min_length=0,
    max_length=40,
)
decoded_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

display(pd.DataFrame(decoded_summaries, columns=["decoded_summaries"]))

,decoded_summaries
0,many businesses and householders were affected...
1,fire alarm went off at the Holiday Inn in Hope...
2,stewards handed a reprimand to f1 team-mate Ki...
3,the 67-year-old is accused of committing the o...
4,a man receiving psychiatric treatment at the c...
5,Gregor Townsend gave a debut to powerhouse win...
6,"Veronica Vanessa Chango-Alverez, 31, was kille..."
7,the 25-year-old was hit by a motorbike during ...
8,rGundogan says he can see the finishing line a...
9,the crash happened about 07:20 GMT at the junc...
